# Data processing

Turn my dictionary into a data frame!

In [1]:
%run Preamble.ipynb

In [2]:
f = open("Data\\initial_data_cleaning.json")
bills = json.load(f)

In [3]:
# Make the bills json file contents into a data frame
bills_df_all = pd.json_normalize(bills)
bills_df_all.columns.values.tolist()

['chamber',
 'bill_type',
 'title',
 'summary',
 'introduced',
 'subject(s)',
 'actions',
 'latest_action',
 'sponsor(s).people',
 'sponsor(s).R',
 'pass_house.passed',
 'pass_house.date',
 'pass_house.votes.people',
 'pass_house.votes.Yea.D',
 'pass_house.votes.Yea.R',
 'pass_house.votes.Nay.D',
 'pass_house.votes.Nay.R',
 'pass_house.votes.Absent.D',
 'pass_senate.passed',
 'pass_senate.date',
 'became_law.signed',
 'became_law.date',
 'sponsor(s).D',
 'pass_house.votes.Absent.R',
 'pass_house.votes',
 'pass_senate.votes.people',
 'pass_senate.votes.Yea.D',
 'pass_senate.votes.Yea.R',
 'pass_senate.votes.Yea.I',
 'pass_house.votes.NV.R',
 'pass_house.votes.NV.D',
 'pass_senate.votes.Nay.R',
 'pass_senate.votes.Absent.R',
 'pass_senate.votes.Nay.D',
 'pass_senate.votes.Nay.I',
 'pass_senate.votes.Absent.D',
 'pass_senate.votes.Absent.I',
 'pass_senate.votes.NV.D',
 'sponsor(s).I']

In [4]:
# Add in a column for voice vote
vv_list = []
for i in range(len(bills)):
    if bills[i]["pass_house"]["votes"] == "Voice vote":
        vv_list.append(1)
    else:
        vv_list.append(0)
bills_df_all["pass_house.votes.people.Voice vote"] = vv_list

In [5]:
# Change sponsors, house votes, senate votes, to columns of lists

print("col_of_lists(\"sponsor(s).people\")")
start_scol = time.time() # for debugging
col_of_lists("sponsor(s).people")
end_scol = time.time() # for debugging
time_elapsed(start_scol, end_scol) # for debugging
print() # for debugging

print("\ncol_of_lists(\"pass_house.votes.people\")")
start_hvcol = time.time() # for debugging
col_of_lists("pass_house.votes.people")
end_hvcol = time.time() # for debugging
time_elapsed(start_hvcol, end_hvcol) # for debugging
print() # for debugging

print("\ncol_of_lists(\"pass_senate.votes.people\")")
start_svcol = time.time() # for debugging
col_of_lists("pass_senate.votes.people")
end_svcol = time.time() # for debugging
time_elapsed(start_svcol, end_svcol) # for debugging

col_of_lists("sponsor(s).people")
Time elapsed = 1.065927505493164 seconds


col_of_lists("pass_house.votes.people")
Time elapsed = 0.2906460762023926 seconds


col_of_lists("pass_senate.votes.people")
Time elapsed = 0.24693584442138672 seconds


In [6]:
# Use the one-hot column of lists function

start_ohcs = time.time() # for debugging
print("one_hot_col(\"sponsor(s).people\")") # for debugging
bills_df_all = one_hot_col("sponsor(s).people")
end_ohcs = time.time() # for debugging
time_elapsed(start_ohcs, end_ohcs) # for debugging
print() # for debugging

start_ohchv = time.time() # for debugging
print("one_hot_col(\"pass_house.votes.people\")") # for debugging
bills_df_all = one_hot_col("pass_house.votes.people")
end_ohchv = time.time() # for debugging
time_elapsed(start_ohchv, end_ohchv) # for debugging
print() # for debugging

start_ohcsv = time.time() # for debugging
print("one_hot_col(\"pass_senate.votes.people\")") # for debugging
bills_df_all = one_hot_col("pass_senate.votes.people")
end_ohcsv = time.time() # for debugging
time_elapsed(start_ohcsv, end_ohcsv) # for debugging
print() # for debugging

start_ohcsj = time.time() # for debugging
print("one_hot_col(\"subject(s)\")") # for debugging
bills_df_all = one_hot_col("pass_senate.votes.people")
end_ohcsj = time.time() # for debugging
time_elapsed(start_ohcsj, end_ohcsj) # for debugging
print() # for debugging

#bills_df_all

one_hot_col("sponsor(s).people")
Time elapsed = 4.011409409840902 minutes

one_hot_col("pass_house.votes.people")
Time elapsed = 9.314754565556845 minutes

one_hot_col("pass_senate.votes.people")
Time elapsed = 1.8303328275680542 minutes

one_hot_col("subject(s)")
Time elapsed = 1.7923264384269715 minutes



In [7]:
# Remove some columns
bills_df_trimmed = bills_df_all.drop(columns = ["title", "summary", "introduced", "subject(s)",
    "latest_action", "sponsor(s).people", "pass_house.date", "pass_house.votes.people", 
    "pass_senate.votes.people", "pass_senate.date", "became_law.date"])

# computing number of columns
print("Number of remaining columns:")
len(bills_df_trimmed.axes[1])

Number of remaining columns:


2377

In [8]:
# One-hot the rest and remove any duplicate columns while we're at it
start_oh = time.time() # for debugging
for column in bills_df_trimmed.columns.values.tolist():
    #print("Column: "+column) # for debugging
    # for duplicated columns?!
    if isinstance(bills_df_trimmed[column], pd.DataFrame):
        bills_df_trimmed[column] = bills_df_trimmed[column].iloc[:,0]
    elif bills_df_trimmed[column].dtype == bool:
        bills_df_trimmed[column] = bills_df_trimmed[column].astype(int)
    elif bills_df_trimmed[column].dtype == object:
        one_hot = pd.get_dummies(bills_df_trimmed[column], dtype=int)
        for col_one_hot in one_hot.columns.values.tolist():
            bills_df_trimmed[column+"."+col_one_hot] = one_hot[col_one_hot]
# Can't seem to do this in the loop            
bills_df_trimmed = bills_df_trimmed.drop(columns=["chamber", "bill_type"])            

end_oh = time.time() # for debugging
time_elapsed(start_oh, end_oh) # for debugging

# Put zeros in the empty entries
bills_df_trimmed = bills_df_trimmed.fillna(0)

# Get rid of redundant Voice vote columns
bills_df_trimmed = bills_df_trimmed.drop(columns=["pass_house.votes.Voice vote", "pass_house.votes"])
bills_df_trimmed

Column: chamber
Column: bill_type
Column: actions
Column: sponsor(s).R
Column: pass_house.passed
Column: pass_house.votes.Yea.D
Column: pass_house.votes.Yea.R
Column: pass_house.votes.Nay.D
Column: pass_house.votes.Nay.R
Column: pass_house.votes.Absent.D
Column: pass_senate.passed
Column: became_law.signed
Column: sponsor(s).D
Column: pass_house.votes.Absent.R
Column: pass_house.votes
Column: pass_senate.votes.Yea.D
Column: pass_senate.votes.Yea.R
Column: pass_senate.votes.Yea.I
Column: pass_house.votes.NV.R
Column: pass_house.votes.NV.D
Column: pass_senate.votes.Nay.R
Column: pass_senate.votes.Absent.R
Column: pass_senate.votes.Nay.D
Column: pass_senate.votes.Nay.I
Column: pass_senate.votes.Absent.D
Column: pass_senate.votes.Absent.I
Column: pass_senate.votes.NV.D
Column: sponsor(s).I
Column: pass_house.votes.people.Voice vote
Column: sponsor(s).people.Joaquin Castro
Column: sponsor(s).people.John Duarte
Column: sponsor(s).people.Judy Chu
Column: sponsor(s).people.Daniel Webster
Colum

Column: sponsor(s).people.Stacey Plaskett
Column: sponsor(s).people.Kevin Mullin
Column: sponsor(s).people.Mark Amodei
Column: sponsor(s).people.Rudolph Yakym III
Column: sponsor(s).people.Brittany Pettersen
Column: sponsor(s).people.Earl Blumenauer
Column: sponsor(s).people.Yvette Clarke
Column: sponsor(s).people.John Hoeven
Column: sponsor(s).people.Amata Radewagen
Column: sponsor(s).people.Brian Mast
Column: sponsor(s).people.Mitch McConnell
Column: sponsor(s).people.Donald Davis
Column: sponsor(s).people.Matt Rosendale
Column: sponsor(s).people.Morgan McGarvey
Column: sponsor(s).people.Donald Norcross
Column: sponsor(s).people.Michelle Fischbach
Column: sponsor(s).people.Nikki Budzinski
Column: sponsor(s).people.John Fetterman
Column: sponsor(s).people.Glenn Thompson
Column: sponsor(s).people.Sheldon Whitehouse
Column: sponsor(s).people.Alex Padilla
Column: sponsor(s).people.Brandon Williams
Column: sponsor(s).people.Jennifer McClellan
Column: sponsor(s).people.Byron Donalds
Column

Column: pass_house.votes.people.Yea.David Trone
Column: pass_house.votes.people.Nay.Steve Cohen
Column: pass_house.votes.people.Nay.Ronny Jackson
Column: pass_house.votes.people.Absent.Norma Torres
Column: pass_house.votes.people.Yea.Max Miller
Column: pass_house.votes.people.Nay.Jacob Laturner
Column: pass_house.votes.people.Absent.Rashida Tlaib
Column: pass_house.votes.people.Absent.Ann Kuster
Column: pass_house.votes.people.Nay.Zachary Nunn
Column: pass_house.votes.people.Absent.Marc Veasey
Column: pass_house.votes.people.Nay.Daniel Goldman
Column: pass_house.votes.people.Absent.Donald Norcross
Column: pass_house.votes.people.Absent.Pete Sessions
Column: pass_house.votes.people.Absent.Darrell Issa
Column: pass_house.votes.people.Nay.Marcy Kaptur
Column: pass_house.votes.people.Absent.Juan Ciscomani
Column: pass_house.votes.people.Yea.Melanie Stansbury
Column: pass_house.votes.people.Nay.Suzan DelBene
Column: pass_house.votes.people.Nay.Debbie Dingell
Column: pass_house.votes.people.

Column: pass_house.votes.people.Yea.Chrissy Houlahan
Column: pass_house.votes.people.Nay.Jake Auchincloss
Column: pass_house.votes.people.Nay.Ken Buck
Column: pass_house.votes.people.Nay.Michael Lawler
Column: pass_house.votes.people.Yea.Seth Magaziner
Column: pass_house.votes.people.Absent.Tony Cardenas
Column: pass_house.votes.people.Absent.Mark Takano
Column: pass_house.votes.people.Nay.Brendan Boyle
Column: pass_house.votes.people.Yea.Michael Guest
Column: pass_house.votes.people.Nay.Jennifer Kiggans
Column: pass_house.votes.people.Yea.Stephanie Bice
Column: pass_house.votes.people.Absent.Yadira Caraveo
Column: pass_house.votes.people.Absent.Jerry Carl
Column: pass_house.votes.people.Absent.Lori Chavez-DeRemer
Column: pass_house.votes.people.Yea.Jared Huffman
Column: pass_house.votes.people.Yea.Jim Jordan
Column: pass_house.votes.people.Yea.Pramila Jayapal
Column: pass_house.votes.people.Absent.Dan Crenshaw
Column: pass_house.votes.people.Absent.Derek Kilmer
Column: pass_house.vote

Column: pass_house.votes.people.Yea.Guy Reschenthaler
Column: pass_house.votes.people.Yea.Neal Dunn
Column: pass_house.votes.people.Absent.Kat Cammack
Column: pass_house.votes.people.Nay.Susan Wild
Column: pass_house.votes.people.Nay.Sydney Kamlager-Dove
Column: pass_house.votes.people.Yea.Daniel Webster
Column: pass_house.votes.people.Nay.Matthew Cartwright
Column: pass_house.votes.people.Absent.Nathaniel Moran
Column: pass_house.votes.people.Nay.Angela Craig
Column: pass_house.votes.people.Absent.John Sarbanes
Column: pass_house.votes.people.Yea.Bonnie Coleman
Column: pass_house.votes.people.Nay.John Sarbanes
Column: pass_house.votes.people.Yea.Aaron Bean
Column: pass_house.votes.people.Absent.Patrick Fallon
Column: pass_house.votes.people.Nay.Julia Brownley
Column: pass_house.votes.people.Absent.Jerrold Nadler
Column: pass_house.votes.people.Absent.French Hill
Column: pass_house.votes.people.Yea.Andy Kim
Column: pass_house.votes.people.Yea.Jason Smith
Column: pass_house.votes.people

...

Column: pass_house.votes.people.Nay.Glenn Grothman
Column: pass_house.votes.people.Yea.Richard Hudson
Column: pass_house.votes.people.Nay.George Santos
Column: pass_house.votes.people.Yea.Alma Adams
Column: pass_house.votes.people.Yea.George Santos
Column: pass_house.votes.people.Nay.Garret Graves
Column: pass_house.votes.people.Absent.Ritchie Torres
Column: pass_house.votes.people.Yea.Marjorie Greene
Column: pass_house.votes.people.Yea.Scott Peters
Column: pass_house.votes.people.Yea.Nancy Pelosi
Column: pass_house.votes.people.Yea.Andrew Clyde
Column: pass_house.votes.people.Absent.Bill Huizenga
Column: pass_house.votes.people.Nay.Richard Neal
Column: pass_house.votes.people.Yea.Russell Fry
Column: pass_house.votes.people.Absent.Ronny Jackson
Column: pass_house.votes.people.Nay.Val Hoyle
Column: pass_house.votes.people.Yea.Mary Gay Scanlon
Column: pass_house.votes.people.Yea.Eric Burlison
Column: pass_house.votes.people.Nay.Josh Harder
Column: pass_house.votes.people.Yea.Jay Obernolt

Column: pass_senate.votes.people.Nay.Mike Lee
Column: pass_senate.votes.people.Absent.Michael Bennet
Column: pass_senate.votes.people.Yea.Joni Ernst
Column: pass_senate.votes.people.Absent.Angus King
Column: pass_senate.votes.people.Absent.Joni Ernst
Column: pass_senate.votes.people.Nay.Michael Bennet
Column: pass_senate.votes.people.Nay.Mazie Hirono
Column: pass_senate.votes.people.Nay.Chuck Grassley
Column: pass_senate.votes.people.Nay.Martin Heinrich
Column: pass_senate.votes.people.Yea.Mike Crapo
Column: pass_senate.votes.people.Yea.Kirsten Gillibrand
Column: pass_senate.votes.people.Yea.Joe Manchin
Column: pass_senate.votes.people.Nay.Cindy Hyde-Smith
Column: pass_senate.votes.people.Nay.Ron Wyden
Column: pass_senate.votes.people.Nay.Brian Schatz
Column: pass_senate.votes.people.Absent.Mike Crapo
Column: pass_senate.votes.people.Absent.Eric Schmitt
Column: pass_senate.votes.people.Nay.Ted Cruz
Column: pass_senate.votes.people.Absent.Mitch McConnell
Column: pass_senate.votes.people

Column: pass_senate.votes.people.Yea.Todd Young
Column: pass_senate.votes.people.Absent.Sheldon Whitehouse
Column: pass_senate.votes.people.Absent.John Fetterman
Column: pass_senate.votes.people.Yea.Sheldon Whitehouse
Column: pass_senate.votes.people.Nay.Jack Reed
Column: pass_senate.votes.people.Yea.Bill Cassidy
Column: pass_senate.votes.people.Absent.Mike Lee
Column: pass_senate.votes.people.Yea.James Risch
Column: pass_senate.votes.people.Yea.Alex Padilla
Column: pass_senate.votes.people.Nay.Kyrsten Sinema
Column: pass_senate.votes.people.Yea.Deb Fischer
Column: pass_senate.votes.people.Nay.Bill Hagerty
Column: pass_senate.votes.people.Nay.Raphael Warnock
Column: pass_senate.votes.people.Nay.Lisa Murkowski
Column: pass_senate.votes.people.Nay.Elizabeth Warren
Column: pass_senate.votes.people.Yea.James Lankford
Column: pass_senate.votes.people.Nay.Sherrod Brown
Column: pass_senate.votes.people.Absent.Richard Durbin
Column: pass_senate.votes.people.Nay.Rick Scott
Column: pass_senate.v

Column: pass_senate.votes.people.Nay.Susan Collins
Column: pass_senate.votes.people.Nay.Edward Markey
Column: pass_senate.votes.people.Nay.Angus King
Column: pass_senate.votes.people.Yea.Jeff Merkley
Column: pass_senate.votes.people.Yea.Lisa Murkowski
Column: pass_senate.votes.people.Yea.Mitt Romney
Column: pass_senate.votes.people.Yea.Charles Schumer
Column: pass_senate.votes.people.Nay.Shelley Capito
Column: pass_senate.votes.people.Yea.Peter Welch
Column: pass_senate.votes.people.Absent.Rick Scott
Column: pass_senate.votes.people.Nay.Thom Tillis
Column: pass_senate.votes.people.Nay.Mike Braun
Column: pass_senate.votes.people.Nay.Robert Casey
Column: pass_senate.votes.people.Yea.Edward Markey
Column: pass_senate.votes.people.Yea.Christopher Murphy
Column: pass_senate.votes.people.Nay.Mike Rounds
Column: pass_senate.votes.people.Nay.Deb Fischer
Column: pass_senate.votes.people.Yea.Gary Peters
Column: pass_senate.votes.people.unanimous
Column: pass_senate.votes.people.Absent.Mark Warne

,actions,sponsor(s).R,pass_house.passed,pass_house.votes.Yea.D,pass_house.votes.Yea.R,pass_house.votes.Nay.D,pass_house.votes.Nay.R,pass_house.votes.Absent.D,pass_senate.passed,became_law.signed,...,pass_senate.votes.people.Nay.John Fetterman,pass_senate.votes.people.Absent.Mitt Romney,pass_senate.votes.people.Absent.Marsha Blackburn,pass_senate.votes.people.Yea.Josh Hawley,chamber.H,chamber.S,bill_type.B,bill_type.CR,bill_type.JR,bill_type.R
0,159,50.0,1,4.0,221.0,203.0,1.0,6.0,0,0,...,0,0,0,0,1,0,1,0,0,0
1,2,10.0,0,0.0,0.0,0.0,0.0,0.0,0,0,...,0,0,0,0,1,0,1,0,0,0
2,2,3.0,0,0.0,0.0,0.0,0.0,0.0,0,0,...,0,0,0,0,1,0,1,0,0,0
3,2,1.0,0,0.0,0.0,0.0,0.0,0.0,0,0,...,0,0,0,0,1,0,1,0,0,0
4,2,4.0,0,0.0,0.0,0.0,0.0,0.0,0,0,...,0,0,0,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15361,1,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0,...,0,0,0,0,0,1,0,0,0,1
15362,1,3.0,0,0.0,0.0,0.0,0.0,0.0,0,0,...,0,0,0,0,0,1,0,0,0,1
15363,1,8.0,0,0.0,0.0,0.0,0.0,0.0,0,0,...,0,0,0,0,0,1,0,0,0,1
15364,1,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0,...,0,0,0,0,0,1,0,0,0,1


In [9]:
# Save to a file
bills_df_trimmed.to_csv("processed_data.csv", index=False)

PermissionError: [Errno 13] Permission denied: 'processed_data.csv'